In [ ]:
import codecs
from nltk import DependencyGraph
from nltk.corpus import stopwords
import re
import pymorphy2
from itertools import tee
import os
from tqdm import tqdm
from collections import Counter
import math
from gensim.models.word2vec import Word2Vec
import gensim
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity as cosine

## Функции для обработки текста:

In [ ]:
## извлечение даты из новости 

def month_to_num(word):
    months = ['января','февраля','марта','апреля','мая','июня','июля','августа','сентября','октября','ноября','декабря']
    res = months.index(word)+1
    if res < 10:
        res = '0'+str(res)
    return(str(res))

def date(news):
    regexp_date = re.compile('Опубликовано\s[0-9]{1,2}\s[А-Яа-я]+\s201\d') 
    date_raw = regexp_date.findall(news)[0].lower().split()[1:]
    date = str(date_raw[0])+'.'+month_to_num(date_raw[1])+'.'+str(date_raw[2])
    return(date)


def reverse(date, mode):
    if mode == 1:
        d,m,y = date.split('.')
        return(str(y)+'.'+str(m)+'.'+str(d))
    else:
        y,m,d = date.split('.')
        return(str(d)+'.'+str(m)+'.'+str(y))
        
           

## извлечение текста новости 
def sents(text):
    s = text.split('\n')
    res = [w.split(' .') for w in s if ((len(w) > 1) and w.split()[0] not in ['Опубликовано', 'Авторы:'])]
    flat_list = [item for sublist in res for item in sublist]
    return(flat_list)



## стоп-слова и замена "компания" на название компании
def clean_lemmas(d):
    global cotags
    stopwords1 = ['это', 'который', 'год', 'день', 'месяц', 'неделя', 'конец', 'начало', 'рубль', 
                  'январь','февраль','март','апрель','май','июнь','июль','август','сентябрь','октябрь','ноябрь','декабрь']
    words = ['компания_NOUN', 'холдинг_NOUN', 'монополия_NOUN', 'госкомпания_NOUN','нефтекомпания_NOUN',
             'энергокомпания_NOUN', 'госкорпорация_NOUN', 'корпорация_NOUN']
    morph = pymorphy2.MorphAnalyzer()
    d1 = {}
    for v in d:
        for group in d[v]:
            lemmas = set()
            for w in d[v][group]:
                p = morph.parse(w)[0]
                if not 'LATN' in p.tag:
                    if w.isalpha() and not (p.normal_form in stopwords.words('russian') or p.normal_form in stopwords1):
                        lemmas.add(p.normal_form+'_'+cotags[p.tag.POS])
                else:
                    lemmas.add(w+'_S')
            
                    
            for w in words:
                if w in lemmas:
                    lemmas.remove(w)
                    lemmas.add(company)
            d[v][group] = lemmas
        
        p = morph.parse(v)[0] 
        d1[p.normal_form+'_'+cotags[p.tag.POS]] = d[v]      
    
    
    return d1

In [ ]:
# разрешение кореференции: замена слов "компания" и аналогичных на потенциальные именованные сущности

def possible_entities(all_news):
    morph = pymorphy2.MorphAnalyzer()
    candidates = {}
    for news in all_news:
        sentences = sents(news)
        for s in sentences:
            for w in s.split()[1:]:
                if w[0].isupper() and len(w) >= 3:
                    n = morph.parse(w)[0].normal_form
                    if n in candidates:
                        candidates[n] += 1
                    else:
                        candidates[n] = 1

    
    return(candidates)



def replace_with_entity(news_list):
    morph = pymorphy2.MorphAnalyzer()
    global named_entities
    global freq_treshold
    global stopwords
    news1 = []
    named_entity = ''
    for sent in news_list:
        if len(sent):
            res = ''
            for w in sent.split():
                n = morph.parse(w)[0]
                if n.normal_form.lower() in named_entities and named_entities[n.normal_form.lower()] > freq_treshold:
                    named_entity = w
                    res += ' ' + w
                elif n.normal_form in stopwords and named_entity != '':
                    try:
                        form = morph.parse(named_entity)[0].inflect({n.tag.case}).word
                        res += ' ' + w + ' ' +form
                    except:
                        res += ' ' + w
                else:
                    res += ' ' + w
            
            news1.append(res)
            
    return(news1)

## Функции для извлечения SVO-троек:

In [ ]:
def extract(triples0):
    sov = {}  
    triples = list(triples0)
    for triple in triples:
        if triple:
            if triple[0][1] == 'VERB':
                sov[triple[0][0]] = {'verb': set(), 'obj': set(), 'subj': set()}
            if (triple[1] == 'xcomp') and triple[0][0] in sov:
                sov[triple[0][0]]['verb'].add(triple[2][0])
                         
    
    for triple in triples:
        if triple:
            if (triple[1] == 'xcomp') and triple[0][0] in sov:
                sov[triple[0][0]]['verb'].add(triple[2][0])
            if triple[1] == 'neg' and triple[0][0] in sov:
                sov[triple[0][0]]['verb'].add('не') 
            if triple[1] == 'nsubj' and triple[0][0] in sov:
                sov[triple[0][0]]['subj'].add(triple[2][0])
            if triple[1] == 'dobj' and triple[0][0] in sov:
                sov[triple[0][0]]['obj'].add(triple[2][0])   

    for verb in sov:
        subj = sov[verb]['subj']
        subj_group = set()
        added = 1
        while (added):
            added = 0
            for triple in triples:
                if (triple[1] == 'dobj' or triple[1] == 'nmod') and triple[0][0] in subj and triple[2][0] not in subj_group:
                    subj_group.add(triple[2][0])
                    triples.remove(triple)
                    added = 1
                if (triple[1] == 'appos' or triple[1] == 'name') and triple[0][0] in subj:
                    subj_group.add(triple[2][0])
                    added = 1
                    triples.remove(triple)
        sov[verb]['subj'].update(subj_group)
                    
          
    for verb in sov:
        obj = sov[verb]['obj']
        obj_group = set()
        added = 1
        while (added):
            added = 0
            for triple in triples:
                if (triple[1] == 'dobj' or triple[1] == 'nmod') and (triple[0][0] in subj or triple[0][0] == verb) and triple[2][0] not in obj_group:
                    triples.remove(triple)
                    obj_group.add(triple[2][0])
                    added = 1
        sov[verb]['obj'].update(obj_group)
    
           
    sov1 = {}
    for v in sov:
        if not 'del' in sov[v]['subj']:
            if not 'не' in sov[v]['verb']:
                sov1[v] = sov[v]
            else:
                sov[v]['verb'].remove('не')
                sov1['не '+v] = sov[v]
    
    return clean_lemmas(sov1)
    



def triples_extraction(path):
    processed_sentences = []
    sentence = []

    for line in codecs.open(path, 'r', 'utf-8'):
        if len(line) == 1:
            processed_sentences.append(sentence)
            sentence = []
        else:
            word = line.split("\t")
            sentence.append(word)

    deps = []
    for sentence in processed_sentences:
        s = u""
        for line in sentence:
            s += u"\t".join(line) + u'\n'
        deps.append(s)
    
    triples = []
    for sent_dep in deps:
        try:
            graph = DependencyGraph(tree_str=sent_dep)
        except:
            pass
        try:
            res = extract(graph.triples())
            triples.append(res)
        except:
            pass
    tr = [d for d in triples if sum([len(v) for v in d.values()])>= 3]
    return(triples)

def sov_by_date(texts_by_date):
    sov = {}
    for d in tqdm(texts_by_date):
        with open ('sents.txt', 'w') as f:
            for i in texts_by_date[d]:
                if len(i) > 1:
                    for j in i:
                        f.write(j)
                        f.write('\n')
        ! cat sents.txt | docker run -a stdout > log.txt --rm -i inemo/syntaxnet_rus > data.conll
        sov[d] = triples_extraction('data.conll')
        
    for d in sov:
        for tr in sov[d]:
            for v1 in tr:
                for cat in tr[v1]:
                    tr[v1][cat] = list(tr[v1][cat])
    return sov

## Извлечение троек:

In [ ]:
# для упорядоченных по компаниям новостей:
# Нужно ввести название компании (на русском языке), тексты новостей по умолчанию лежат в папке utilities.
company = input('Название компании: ')

companies = {'Алроса': 'alrosa',
             'Газпром': 'gazprom',
             'Интер РАО': 'interrao',
             'Лукойл': 'lukoil',
             'Норникель': 'nornikel',
             'Новатэк': 'novatek',
             'Росатом': 'rosatom',
             'Русснефть': 'russneft',
             'Татнефть': 'tatneft',
             'Транснефть': 'transeft',
             'test':'test'
            }


with open('utilities/%s.txt' % companies[company]) as f:
        raw = f.read()
        
raw = raw.replace('.', ' .').replace(',', ' ,')
raw = re.sub(r'[«»"]','', raw)
news = raw.split('--------------------------------------------------------------------------')

news = list(set(news))
print('Новостей: ', len(news))


texts_by_date = {}
for n in news:
    try:
        d = date(n)
    except:
        pass
    s = sents(n)
    if d in texts_by_date:
        texts_by_date[d].append(s)
    else:
        texts_by_date[d]=[s] 

In [ ]:
# для неупорядоченных по компаниям новостей
        
with open('path_to_txt') as f:
        raw = f.read()
        
        
        
raw = raw.replace('.', ' .').replace(',', ' ,')
raw = re.sub(r'[«»"]','', raw)
news = raw.split('--------------------------------------------------------------------------')

news = list(set(news))
print('Новостей: ', len(news))

freq_treshold = 100 # пороговое значение: если слово, начинающееся с заглавной буквы, встретилось в тексте больше freq_treshold раз,
                    # то оно считается кандидатом в "именованную сущностью"

# этот список нужно расширить :(
stopwords = ['компания', 'холдинг', 'монополия', 'госкомпания','нефтекомпания','энергокомпания', 'госкорпорация', 'корпорация']
named_entities = possible_entities(news)

texts_by_date = {}
for n in news:
    try:
        d = date(n)
    except:
        pass
    s = replace_with_entity(sents(n))
    if d in texts_by_date:
        texts_by_date[d].append(s)
    else:
        texts_by_date[d]=[s]       

In [ ]:
# извлечение SVO-троек:

data = sov_by_date(texts_by_date)

##  Построение эмбеддингов:

In [ ]:
def w2v_tag(lemma):    
    cotags = {'ADJF':'A', # pymorphy2: rusvectores
                'ADJS' : 'A', 
                'ADVB' : 'ADV', 
                'COMP' : 'A',
                'CONJ' : 'CONJ',
                'GRND' : 'V',
                'NUMR' : 'NUM',
                'INFN' : 'V', 
                'NOUN' : 'S', 
                'PRED' : 'V', 
                'PRTF' : 'A', 
                'PRTS' : 'V', 
                'VERB' : 'V',
                'NPRO' : 'V',
                'PRCL' : 'PART',
                'PREP': 'PR'} 
    word, tag = lemma.split('_')
    return (word + '_' + cotags[tag])

In [ ]:
## функции для вычисления tf-idf по всему корпусу текстов



def compute_tfidf(corpus):
    def compute_tf(text):
        tf_text = Counter(text)
        for i in tf_text:
            tf_text[i] = tf_text[i]/float(len(text))
        return tf_text
    def compute_idf(word, corpus):
        return math.log10(len(corpus)/sum([1.0 for i in corpus if word in i]))

    documents_list = []
    for text in corpus:
        tf_idf_dictionary = {}
        computed_tf = compute_tf(text)
        for word in computed_tf:
            tf_idf_dictionary[word] = computed_tf[word] * compute_idf(word, corpus)
        documents_list.append(tf_idf_dictionary)
    return documents_list

def sents_by_words(text):
    def date(news):
        try:
            regexp_date = re.compile('Опубликовано\s[0-9]{1,2}\s[А-Яа-я]+\s201\d') 
            date_raw = regexp_date.findall(news)[0].lower().split()[1:]
            date = str(date_raw[0])+'.'+month_to_num(date_raw[1])+'.'+str(date_raw[2])
            return(date)
        except:
            return '00.00.0000'

    s = text.split('\n')
    res = [w.split(' .') for w in s if ((len(w) > 1)) and w.split()[0]]
    flat_list = [item for sublist in res for item in sublist]
    l = 'Дата:'+str(date(text)) +' ' + ' '.join([s for s in flat_list])
    words = ' '.join([morph.parse(s)[0].normal_form for s in l.split()]).split()
    return(words)

In [ ]:
## вычисляем tf-idf для всего корпуса сразу: соответствующие веса хранятся в словаре по датам

corpus = []
for n in news:
    corpus.append(sents_by_words(n))
    
tf_idf = compute_tfidf(corpus)

tf_idf_by_date = {}

for n in tf_idf:
    for key in n:
        if key.startswith('дата:'):
            d = key[5:]
    if d not in tf_idf_by_date:
        tf_idf_by_date[d] = n
    else:
        s = dict(Counter(tf_idf_by_date[d])+Counter(n))
        tf_idf_by_date[d] = s

In [ ]:
## предобученная модель Rusvectores

model = gensim.models.KeyedVectors.load_word2vec_format("news_mystem_skipgram_1000_20_2015.bin.gz", binary=True)

In [ ]:
## перевод SVO-троек в эмбеддинги событий

triples_vec = {}


for date in data:
    tf_idf = tf_idf_by_date[date]
    embedding = []
    for tr in data[date]:       
        for verb in tr:
            verb_vec = np.zeros(1000)
            if tr[verb]['verb'] == []:
                try:
                    for v in verb.split():
                        try:
                            tfidf = tf_idf[v.split('_')[0]]
                        except:
                            tfidf = 0.1
                        verb_vec += model[w2v_tag(v)]*tfidf
                except:
                    pass
            else:
                try:
                    try:
                        tfidf1 = tf_idf[verb.split('_')[0]]     
                    except:
                        tfidf1 = 0.1
                    try:
                        tfidf2 = tf_idf[tr[verb]['verb'].split('_')[0]]
                    except:
                        tfidf2 = 0.1
        
                    verb_vec = model[w2v_tag(verb)]*tfidf1 + model[w2v_tag(tr[verb]['verb'])]*tfidf2
                except:
                    try:
                        try:
                            tfidf = tf_idf[tr[verb]['verb'].split('_')[0]]     
                        except:
                            tfidf = 0.1
                            
                        verb_vec = model[w2v_tag(tr[verb]['verb'])] * tfidf
                        for v in verb.split():
                            try:
                                tfidf = tf_idf[v.split('_')[0]] 
                            except:
                                tfidf = 0.1
                            verb_vec += model[w2v_tag(v)] * tfidf
                    except:
                        pass
            
            
            obj_vec = np.zeros(1000)
            for w in tr[verb]['obj']:
                if w2v_tag(w) in model:
                    try:
                        tfidf = tf_idf[w.split('_')[0]]
                    except:
                        tfidf = 0.1    
                    obj_vec += model[w2v_tag(w)] * tfidf
                
            subj_vec = np.zeros(1000)
            for w in tr[verb]['subj']:
                if w2v_tag(w) in model:
                    try:
                        tfidf = tf_idf[w.split('_')[0]]     
                    except:
                        tfidf = 0.1
                    subj_vec += model[w2v_tag(w)] * tfidf
            
            average_emb = (verb_vec + subj_vec + obj_vec) / 3
            if reverse(date,1) not in triples_vec:
                triples_vec[reverse(date,1)] = [average_emb]
            else:
                triples_vec[reverse(date,1)].append(average_emb)

                
for date in triples_vec:
    if len(triples_vec[date]) != 1:
        triples_vec[date] = np.array(triples_vec[date]).mean(axis=0)
    else:
        triples_vec[date] = triples_vec[date][0]

for date in triples_vec:
    if len(triples_vec[date]) == 1:
        triples_vec[date] = triples_vec[date][0]
                            
            
                    

In [ ]:
df = pd.DataFrame.from_dict([triples_vec]).transpose().sort_index()
df.head()

In [ ]:
d = ['2017.05.25','2017.05.15','2017.03.29','2017.01.28']

for d1 in d:
    for d2 in d:
        c = cosine(triples_vec[d1].reshape(1,-1), triples_vec[d2].reshape(1,-1))[0]
        print(d1, ' ', d2, ' ', c)